# 第4章 朴素贝叶斯
1．朴素贝叶斯法是典型的生成学习方法。生成方法由训练数据学习联合概率分布 $P(X,Y)$，然后求得后验概率分布$P(Y|X)$。具体来说，利用训练数据学习$P(X|Y)$和$P(Y)$的估计，得到联合概率分布：

$$P(X,Y)＝P(Y)P(X|Y)$$
概率估计方法可以是极大似然估计或贝叶斯估计。

2．朴素贝叶斯法的基本假设是条件独立性，

$$\begin{aligned} P(X=x | Y=c_{k} )=P\left(X^{(1)}=x^{(1)}, \cdots, X^{(n)}=x^{(n)} | Y=c_{k}\right) \\ =\prod_{j=1}^{n} P\left(X^{(j)}=x^{(j)} | Y=c_{k}\right) \end{aligned}$$
这是一个较强的假设。由于这一假设，模型包含的条件概率的数量大为减少，朴素贝叶斯法的学习与预测大为简化。因而朴素贝叶斯法高效，且易于实现。其缺点是分类的性能不一定很高。

3．朴素贝叶斯法利用贝叶斯定理与学到的联合概率模型进行分类预测。

$$P(Y | X)=\frac{P(X, Y)}{P(X)}=\frac{P(Y) P(X | Y)}{\sum_{Y} P(Y) P(X | Y)}$$
将输入$x$分到后验概率最大的类$y$。

$$y=\arg \max _{c_{k}} P\left(Y=c_{k}\right) \prod_{j=1}^{n} P\left(X_{j}=x^{(j)} | Y=c_{k}\right)$$
后验概率最大等价于0-1损失函数时的期望风险最小化。

#### 模型：

·高斯模型

·多项式模型

·伯努利模型

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

from collections import Counter
import math

In [5]:
# data
def create_data():
    iris = load_iris()
    df = pd.DataFrame(iris.data, columns=iris.feature_names)
    df['label'] = iris.target
    df.columns = ['sepal length', 'sepal width', 'petal length', 'petal width', 'label']
    data = np.array(df.iloc[:100, :])
    # print(data)
    return data[:,:-1], data[:,-1]

In [6]:
X, y = create_data()     # X是'sepal length', 'sepal width', 'petal length', 'petal width'；y是‘label’
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [7]:
# 计算先验概率的0和1出现的概率，即𝑃(𝑌=𝑐𝑘)，用于后期计算𝑦=arg max 𝑃(𝑌=𝑐𝑘)∏𝑃(𝑋𝑗=𝑥(𝑗)|𝑌=𝑐𝑘)
total = Counter(y_train).most_common()
prob_1 = total[0][1]/(total[0][1]+total[1][1])
prob_0 = total[1][1]/(total[0][1]+total[1][1])
prob = {0:prob_0, 1:prob_1}

## GaussianNB 高斯朴素贝叶斯
特征的可能性被假设为高斯分布

概率密度函数：$$P(x_i | y_k)=\frac{1}{\sqrt{2\pi\sigma^2_{yk}}}exp(-\frac{(x_i-\mu_{yk})^2}{2\sigma^2_{yk}})$$

数学期望(mean)：$\mu$

方差：$\sigma^2=\frac{\sum(X-\mu)^2}{N}$

In [8]:
class NaiveBayes:
    def __init__(self):
        self.model = None

    # 数学期望
    @staticmethod
    def mean(X):
        return sum(X) / float(len(X))

    # 标准差（方差）
    def stdev(self, X):
        avg = self.mean(X)
        return math.sqrt(sum([pow(x - avg, 2) for x in X]) / float(len(X)))

    # 概率密度函数
    def gaussian_probability(self, x, mean, stdev):
        exponent = math.exp(-(math.pow(x - mean, 2) /
                              (2 * math.pow(stdev, 2))))
        return (1 / (math.sqrt(2 * math.pi) * stdev)) * exponent

    # 计算X_train每一列的均值和标准差
    def summarize(self, train_data):
        summaries = [(self.mean(i), self.stdev(i)) for i in zip(*train_data)]        
        #  zip(*) 号操作符，可以将元组解压为列表，列表中包含四组数据，分别是'sepal length', 'sepal width', 'petal length', 'petal width'
        return summaries

    # 分类别求出数学期望和标准差
    def fit(self, X, y):
        labels = list(set(y))      # set()产生一个无序不重复的列表，这里鸢尾花数据的y是0、1、2
        data = {label: [] for label in labels}
        for f, label in zip(X, y):
            data[label].append(f)    # data是一个字典，键是“0,1,2”，值是相对应的'sepal length', 'sepal width', 'petal length', 'petal width'
        self.model = {
            label: self.summarize(value) for label, value in data.items()    # 新设立一个model字典，键是“0,1,2”，值是四列数据的均值和方差
        }
        return 'gaussianNB train done!'

    # 计算概率
    def calculate_probabilities(self, input_data):
        # summaries:{0.0: [(5.0, 0.37),(3.42, 0.40)], 1.0: [(5.8, 0.449),(2.7, 0.27)]}
        # input_data:[1.1, 2.2]
        probabilities = {}
        for label, value in self.model.items():
            probabilities[label] = prob[label]        # 原代码是probabilities[label] = prob[label](已做修改)。为什么默认𝑃(𝑌=𝑐𝑘)=1？
            for i in range(len(value)):
                mean, stdev = value[i]
                probabilities[label] *= self.gaussian_probability(      # 对应公式：𝑦=arg max 𝑃(𝑌=𝑐𝑘)∏𝑃(𝑋𝑗=𝑥(𝑗)|𝑌=𝑐𝑘)
                                        input_data[i], mean, stdev)
        return probabilities

    # 预测类别
    def predict(self, X_test):
        # {0.0: 2.9680340789325763e-27, 1.0: 3.5749783019849535e-26}
        label = sorted(
            self.calculate_probabilities(X_test).items(),
            key=lambda x: x[-1])[-1][0]                # 取最大的概率对应的label
        return label

    # 预测结果的正确率
    def score(self, X_test, y_test):
        right = 0
        for X, y in zip(X_test, y_test):
            label = self.predict(X)
            if label == y:
                right += 1

        return right / float(len(X_test))

In [9]:
model = NaiveBayes()
model.fit(X_train, y_train)

'gaussianNB train done!'

In [10]:
print(model.predict([4.4,  3.2,  1.3,  0.2]))

0.0


In [11]:
model.score(X_test, y_test)

1.0

## scikit-learn实例

In [12]:
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection  import cross_val_score

In [15]:
clf = GaussianNB()
clff = clf.fit(X_train, y_train)

In [20]:
clf.score(X_test, y_test)
scores = cross_val_score(clf,X_train, y_train,cv=10)
print("Accuracy:%.3f"%scores.mean())

Accuracy:1.000


In [21]:
clff.predict([[4.4,  3.2,  1.3,  0.2]])

array([0.])

### 还有伯努利模型和多项式模型

In [13]:
from sklearn.naive_bayes import BernoulliNB, MultinomialNB    # 伯努利模型和多项式模型